# Programa para generar planes de mantenimiento, a partir del URL, o desde los archivos CSV descargados

Modificaciones realizadas 12/2025
* Generacion a partir de Lineas Alta Demanda
* Lineas Baja Demanda




## Cargar librerias necesarias

In [130]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# from pandas import DataFrame, Series

## Cargar hoja Equipos-Planes

In [131]:

## Importar CSV de Estructura funcional Ubicacion
import pandas as pd
import re
import os

# This function will convert the url to a download link
def convert_gsheets_url(u):
    try:
        worksheet_id = u.split('#gid=')[1]
    except:
        # Couldn't get worksheet id. Ignore it
        worksheet_id = None
    u = re.findall('https://docs.google.com/spreadsheets/d/.*?/',u)[0]
    u += 'export'
    u += '?format=csv'
    if worksheet_id:
        u += '&gid={}'.format(worksheet_id)
    return u

URL = 'https://docs.google.com/spreadsheets/d/1yOaSeqRBr1FW6tvFMi_Y-s4011cKBoyiWU5dTMlujrU/edit?gid=1115106678#gid=1115106678'     # introducir desde consola
csv_raw = convert_gsheets_url(URL)
print(csv_raw)


# Si existe el CSV importarlo, sino cargar de la URL
filename = "csv/Equipos-PlanesLBD.csv"

if os.path.exists(filename):
    print("Leyendo desde Archivo")
    df_equipos = pd.read_csv(filename)    
else:
    print("Leyendo desde URL")
    df_equipos = pd.read_csv(csv_raw,header=3)
    df_equipos.to_csv(filename,index=False)
print(df_equipos.shape)

https://docs.google.com/spreadsheets/d/1yOaSeqRBr1FW6tvFMi_Y-s4011cKBoyiWU5dTMlujrU/export?format=csv&gid=1115106678
Leyendo desde Archivo
(2751, 111)


## Importar maestro de actividades

In [132]:
## Importar CSV de Planes de Mantto
URL = 'https://docs.google.com/spreadsheets/d/1yOaSeqRBr1FW6tvFMi_Y-s4011cKBoyiWU5dTMlujrU/edit?gid=1199302294#gid=1199302294'     # Introducir desde consola
csv_raw = convert_gsheets_url(URL)
print(csv_raw)
# Si existe el CSV importarlo, sino cargar de la URL
filename = "csv/MaestroActividadesLBD.csv"
if os.path.exists(filename):
    print("Leyendo desde Archivo")
    df_planes = pd.read_csv(filename)        
else:
    print("Leyendo desde URL")
    df_planes = pd.read_csv(csv_raw,header=3)
    df_planes.to_csv(filename,index=False)
print(df_planes.shape)
# Eliminar si tiene valores nulos
df_planes = df_planes [df_planes .Plan.notnull()]
print(df_planes.shape)

https://docs.google.com/spreadsheets/d/1yOaSeqRBr1FW6tvFMi_Y-s4011cKBoyiWU5dTMlujrU/export?format=csv&gid=1199302294
Leyendo desde Archivo
(1577, 43)
(1576, 43)


In [133]:
print (f"Existen {df_planes.Accion.isnull().sum()} valores nulos en la columna Accion, filtrar los no nulos")
df_update = df_planes[~df_planes.Accion.isna()]     # Filtrar solo las columnas de dfPlanes que no tienen accion

Existen 157 valores nulos en la columna Accion, filtrar los no nulos


In [134]:
print(f"Eliminando :{df_update[df_update ['Especialidad']=='BASADA EN CONDICION']['Especialidad'].count()} tareas basadas en condicicion")
# eliminar las tareas basadas en condicion, #TODO: Analizar este cambio. }
df_update = df_update[df_update ['Especialidad']!='BASADA EN CONDICION']

Eliminando :149 tareas basadas en condicicion


In [135]:
# En el dataframe de planes, en la fila del plan se coloca si tiene alguna frecuencia al inicio del plan
gb = df_update.groupby('Plan',sort=False).any()
gb = gb.reset_index()
# Concatenamos los dos dataframes por filas
df_planes = pd.concat([gb,df_update], axis=0, ignore_index=True)

In [136]:
# En caso no este instalado, no se puede manipular excel
#!pip install openpyxl

In [137]:
# Hacemos un join de los dos DataFrames por la izquierda
df_merged = pd.merge(df_equipos, df_planes, how='left', left_on= 'Plan', right_on= 'Plan',suffixes=('_equipo', '_plan'))
# aplicar una funcion en una columna de un dataframe dependiendo si la columna que actualizamos no estan dentro de ['Actividad','Tarea'], si no copiar el valor de la columna 'tipo_equipo', si tiene valor, mantener el valor de la columna 'tipo_plan'
df_merged['Tipo_plan']=df_merged.apply(lambda row: row['Tipo_equipo'] if  row['Tipo_plan']  not in ['Actividad','Tarea']  else  row ['Tipo_plan'] , axis='columns')
# realizar una funcion apply en el dataframe df_merge que evalue el contenido de la columna 'Tipo_Plan', si su valor es 'Componente' que vacie el contenido de la columna 'Accion' y la columna 'Actividad', de lo contrario que mantenga su valor
df_merged['Accion'] = df_merged.apply(lambda row : '' if row['Tipo_plan'] == 'Componente' else row['Accion'],axis='columns')

df_merged['Actividad'] = df_merged.apply(lambda row : '' if row['Tipo_plan'] == 'Componente' else row['Actividad'],axis='columns')

#Exportar dataframe a Excel para visualizacion 
df_merged.to_excel('xls/merged.xlsx') # 12Mb el archivo, dificil de analizar

398

In [ ]:

print(f"Se identifican {len(df_merged[df_merged['Tipo_plan']=='Equipo'].index)} equipos a actualizar")
# EQUIPOS
index = df_merged[df_merged['Tipo_plan']=='Equipo'].index.to_list()
#print(len(index))
# obtener las columnas que se desean actualizar
cols = ['Sist','Subs','Equ','Comp','D', 'S', '2S', 'M','MC', '2M', 'T', 'SE', '8M', 'A', '1.5A', '2A', '3A', '4A', '5A', '6A', '8A', '10A', '1000','1300','1800','6000', '22500', '40000', '55000']
m_cols = df_merged.columns.to_list()
update_cols = [i for i in m_cols if i not in cols]
copy_cols = update_cols + ['Sist','Subs','Equ','Comp']


Se identifican 398 equipos a actualizar
Se exportaran las siguientes columnas para actualizar los equipos:
['id_equipo', 'Equipo', 'Componente', 'Parte', 'Tipo_equipo', 'Criticidad', 'Sigla', 'Tag', 'Plan', 'AMEF', 'VE', 'VE-S2-R1', 'VE-S2-I1', 'VE-S2-M1', 'VE-S2-TO', 'VE-S2-CT', 'VE-S2-LAA', 'VE-S1-M1', 'VE-S1-R1', 'VE-S1-TO', 'VE-S1-CT', 'VE-S1-LAA', 'VE-LIN-VH', 'VE-EDF-EIR', 'VE-EDF-EOB', 'VE-EDF-EAO', 'VE-EDF-ELB', 'VE-S2-I1-TMC', 'BL', 'BL-S1-M1', 'BL-S1-R1', 'BL-S1-TO', 'BL-S1-CT', 'BL-S1-LAA', 'BL-S2-R1', 'BL-S2-I1', 'BL-S2-M1', 'BL-S2-TO', 'BL-S2-CT', 'BL-S2-LAA', 'BL-LIN-VH', 'BL-EDF-ESJ', 'BL-EDF-ETR', 'BL-EDF-EBH', 'BL-EDF-EVI', 'BL-S1-M1-TMC', 'BL-S2-M1-TMC', 'NA', 'NA-S1-R1', 'NA-S1-I1', 'NA-S1-M1', 'NA-S1-TO', 'NA-S1-CT', 'NA-S1-LAA', 'NA-S2-M1', 'NA-S2-R1', 'NA-S2-TO', 'NA-S2-CT', 'NA-S2-LAA', 'NA-LIN-VH', 'NA-EDF-ECN', 'NA-EDF-EAR', 'NA-EDF-EPE', 'NA-EDF-EVI', 'NA-S1-R1-TMC', 'CE', 'CE-S1-R1', 'CE-S1-M1', 'CE-S1-TO', 'CE-S1-CT', 'CE-S1-LAA', 'CE-S2-R1', 'CE-S2-M1', 'CE

In [ ]:

print("Se exportaran las siguientes columnas para actualizar los equipos:")
print (copy_cols)
df_g = df_merged[df_merged['Tipo_plan'].isin(['Equipo','Componente'])][cols]


In [113]:
# Agrupar por Equ
gb= df_g.groupby(['Equ'],sort=False).any().reset_index()
gb

,Equ,Sist,Subs,Comp,D,S,2S,M,MC,2M,...,6A,8A,10A,1000,1300,1800,6000,22500,40000,55000
0,MEPA,True,True,True,True,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
1,MEPR,True,True,True,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
2,AMR,True,True,True,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,RPR,True,True,True,True,False,True,False,False,False,...,False,False,False,True,False,False,True,False,False,False
4,RAR,True,True,True,True,False,False,False,False,False,...,False,False,False,True,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392,BALT,True,True,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
393,RBALE,True,True,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
394,TDRU,True,True,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
395,ELECT,True,True,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [121]:
index

[2,
 29,
 41,
 57,
 73,
 108,
 119,
 137,
 204,
 235,
 269,
 295,
 343,
 397,
 547,
 564,
 659,
 678,
 696,
 714,
 732,
 787,
 820,
 898,
 919,
 971,
 987,
 1012,
 1028,
 1054,
 1079,
 1107,
 1156,
 1199,
 1242,
 1297,
 1347,
 1402,
 1475,
 1542,
 1545,
 1613,
 1658,
 1703,
 1727,
 1788,
 1799,
 1824,
 1829,
 1851,
 1873,
 1884,
 1905,
 1918,
 1921,
 1927,
 1930,
 1944,
 1947,
 1960,
 1994,
 2009,
 2013,
 2031,
 2036,
 2049,
 2060,
 2072,
 2131,
 2153,
 2155,
 2169,
 2282,
 2291,
 2300,
 2321,
 2330,
 2335,
 2340,
 2357,
 2362,
 2371,
 2376,
 2381,
 2402,
 2413,
 2424,
 2435,
 2439,
 2443,
 2447,
 2529,
 2611,
 2693,
 2775,
 2857,
 2939,
 3021,
 3103,
 3185,
 3267,
 3349,
 3431,
 3513,
 3595,
 3677,
 3759,
 3841,
 3923,
 4005,
 4087,
 4169,
 4251,
 4333,
 4415,
 4497,
 4579,
 4661,
 4743,
 4825,
 4907,
 4989,
 5071,
 5153,
 5235,
 5317,
 5399,
 5481,
 5563,
 5645,
 5727,
 5809,
 5891,
 5973,
 6055,
 6137,
 6219,
 6301,
 6383,
 6465,
 6547,
 6629,
 6711,
 6793,
 6875,
 6957,
 7039,
 712

In [120]:
len(index)

398

In [116]:
gb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397 entries, 0 to 396
Data columns (total 29 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Equ     397 non-null    object
 1   Sist    397 non-null    bool  
 2   Subs    397 non-null    bool  
 3   Comp    397 non-null    bool  
 4   D       397 non-null    bool  
 5   S       397 non-null    bool  
 6   2S      397 non-null    bool  
 7   M       397 non-null    bool  
 8   MC      397 non-null    bool  
 9   2M      397 non-null    bool  
 10  T       397 non-null    bool  
 11  SE      397 non-null    bool  
 12  8M      397 non-null    bool  
 13  A       397 non-null    bool  
 14  1.5A    397 non-null    bool  
 15  2A      397 non-null    bool  
 16  3A      397 non-null    bool  
 17  4A      397 non-null    bool  
 18  5A      397 non-null    bool  
 19  6A      397 non-null    bool  
 20  8A      397 non-null    bool  
 21  10A     397 non-null    bool  
 22  1000    397 non-null    bo

In [114]:
gb['id'] = index

ValueError: Length of values (398) does not match length of index (397)

In [68]:


gb.set_index('id',inplace=True)
#print (gb)

#TODO: Necesario mejorar el rendimiento del algoritmo aparecen mensajes de warning
#gb = pd.concat([gb,df[cols]],axis = 1)
gb[copy_cols] = df_merged.loc[gb.index,copy_cols]
df_merged.loc[gb.index,:] = gb

# Exportar excel para visualizacion
#df.head(302).to_excel('xls/df.xlsx')

# Subsistema
index = df_merged[df_merged['Tipo_plan']=='Subsistema'].index.to_list()
print((index))
df_g = df_merged[df_merged['Tipo_plan'].isin(['Equipo','Subsistema'])][cols]

# Agrupar por Subs
gb= df_g.groupby(['Subs'],sort=False).any().reset_index()
gb['id'] = index
gb.set_index('id',inplace=True)
# dar formato para copiar
gb[copy_cols] = df_merged.loc[gb.index,copy_cols]
df_merged.loc[gb.index,:] = gb

#df.head(302).to_excel('xls/df.xlsx')

############# Sistema
index = df_merged[df_merged['Tipo_plan']=='Sistema'].index.to_list()
#print(index)
df_g = df_merged[df_merged['Tipo_plan'].isin(['Sistema','Subsistema'])][cols]
# Agrupar por Sist
gb= df_g.groupby(['Sist'],sort=False).any().reset_index()
gb['id'] = index
gb.set_index('id',inplace=True)
# dar formato para copiar
gb[copy_cols] = df_merged.loc[gb.index,copy_cols]
df_merged.loc[gb.index,:] = gb

# TODO: Reemplazar los valores en la columna 'Parada' por 'null' cuando 'Tipo_plan' sea igual a 'Componente'
df_merged.loc[df_merged['Tipo_plan'] == 'Componente', ['Parada','Relevancia','Especialidad']] = None

# TODO: Es necesario que los valores asignados como tareas se quite la relevancia para que no sean tomados en cuenta en la ponderación del check
# Filtrar las filas donde Tipo_plan es igual a 'Tarea' y establecer la columna 'relevancia' a None
df_merged.loc[df_merged['Tipo_plan'] == 'Tarea', ['Relevancia','Parada','Especialidad']] = None

df_merged.head(100).to_excel('xls/df_merged.xlsx')
#print(df)




ValueError: Length of values (398) does not match length of index (397)

In [9]:
# Abrir archivo json

import json
with open('./csv/Lineas.json') as f:
    diccionario_lineas = json.load(f)

frecuencias=['D', 'S', 'M', '2M', 'T', '4M', 'SE', '8M', 'A', '1.5A', '2A', '3A', '4A', '5A', '6A', '9A', '10A', '1000', '6000', '22500', '40000', '55000']

# df_merged[ ['id', 'Cod', 'Accion', 'Actividad', 'Tipo_plan', 'Parada', 'Relevancia', 'Especialidad']+frecuencias]
head = ['id', 'Cod', 'Equipo','Componente', 'Accion', 'Actividad', 'Tipo_plan', 'Parada', 'Relevancia', 'Especialidad'] 

version = "_v2_1_2023"  # importar desde consola

for linea in diccionario_lineas['Lineas']:
    print (linea['Linea'])
    print ( head + frecuencias + linea['Columnas'])       
    # exportar plan maestro de linea
    df_linea = df_merged[head + frecuencias + linea['Columnas']]

    # A nivel de componente, solamente mantiene el primer valor, si encuentra mas adelante coloca ''
    # df_linea.loc[df_linea.duplicated(subset=['Componente']), 'Componente'] = '' 

    #Filtrar solamente los datos de la columna de la linea correspondiente, no todas las demas columnas
    df_linea = df_linea[df_linea[linea['Linea']] == True]
    ## Exportar Excel de Estructura - Planes    
    df_linea.to_excel(f"xls/{linea['Linea']}{version}.xlsx" , index= False)     
    print(df_linea.shape)        
    # break #TODO: En caso solo se genera de 1 linea

RO
['id', 'Cod', 'Equipo', 'Componente', 'Accion', 'Actividad', 'Tipo_plan', 'Parada', 'Relevancia', 'Especialidad', 'D', 'S', 'M', '2M', 'T', '4M', 'SE', '8M', 'A', '1.5A', '2A', '3A', '4A', '5A', '6A', '9A', '10A', '1000', '6000', '22500', '40000', '55000', 'RO', 'RO-S2-R1', 'RO-S2-M1', 'RO-S1-M1', 'RO-S1-R1']
(1237, 37)
AM
['id', 'Cod', 'Equipo', 'Componente', 'Accion', 'Actividad', 'Tipo_plan', 'Parada', 'Relevancia', 'Especialidad', 'D', 'S', 'M', '2M', 'T', '4M', 'SE', '8M', 'A', '1.5A', '2A', '3A', '4A', '5A', '6A', '9A', '10A', '1000', '6000', '22500', '40000', '55000', 'AM', 'AM-S1-R1', 'AM-S1-M1', 'AM-S1-R2', 'AM-S2-R1', 'AM-S2-M1']
(1639, 38)
VE
['id', 'Cod', 'Equipo', 'Componente', 'Accion', 'Actividad', 'Tipo_plan', 'Parada', 'Relevancia', 'Especialidad', 'D', 'S', 'M', '2M', 'T', '4M', 'SE', '8M', 'A', '1.5A', '2A', '3A', '4A', '5A', '6A', '9A', '10A', '1000', '6000', '22500', '40000', '55000', 'VE', 'VE-S2-R1', 'VE-S2-I1', 'VE-S2-M1', 'VE-S1-M1', 'VE-S1-R1']
(1628, 38)
A

In [24]:
# Lineas de Alta demanda
""" print("=========================")
print("Lineas de Alta Demanda")
URL = 'https://docs.google.com/spreadsheets/d/1oUHkuKpHtuhMirNW6SvAQ4A0ns5PZs71iZ_WFXZHNn8/edit?gid=1115106678#gid=1115106678'     # introducir desde consola
csv_raw = convert_gsheets_url(URL)
print(csv_raw)

# Si existe el CSV importarlo, sino cargar de la URL
filename = "csv/Equipos-PlanesLAD.csv"

if os.path.exists(filename):
    print("Leyendo desde Archivo")
    df_equipos2 = pd.read_csv(filename)    
else:
    print("Leyendo desde URL")
    df_equipos2 = pd.read_csv(csv_raw,header=3)
    df_equipos2.to_csv(filename,index=False)
print(df_equipos2.shape)
 """
#print("Unir ambos dataframes")
# Unir ambos dataframes
#df_equipos = pd.concat([df_equipos, df_equipos2], ignore_index=True)

' print("=========================")\nprint("Lineas de Alta Demanda")\nURL = \'https://docs.google.com/spreadsheets/d/1oUHkuKpHtuhMirNW6SvAQ4A0ns5PZs71iZ_WFXZHNn8/edit?gid=1115106678#gid=1115106678\'     # introducir desde consola\ncsv_raw = convert_gsheets_url(URL)\nprint(csv_raw)\n\n# Si existe el CSV importarlo, sino cargar de la URL\nfilename = "csv/Equipos-PlanesLAD.csv"\n\nif os.path.exists(filename):\n    print("Leyendo desde Archivo")\n    df_equipos2 = pd.read_csv(filename)    \nelse:\n    print("Leyendo desde URL")\n    df_equipos2 = pd.read_csv(csv_raw,header=3)\n    df_equipos2.to_csv(filename,index=False)\nprint(df_equipos2.shape)\n '